In [1]:
# A little thing to suppress warnings I know will come up...comment off for testing
#import warnings
#warnings.filterwarnings("ignore", category=RuntimeWarning)
#import math

In [8]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
from scipy.interpolate import interp1d

c,gc = 2.9979*1e8, 6.6726*1e-11
MeV = 1.602*1e-13
Msol = 1.98892*1e30
rs = Msol*gc/c**2/1000   # Schwarzschild radius 1M
# Convert nuclear to astro units:
nuc_to_astro = rs*MeV*1e54/Msol/c**2

def read_eos(pathname):
    """
    Read in equation of state information from tabulated format.
    This is written assuming the following order:
    0: number density [fm-3]
    1: pressure [Mev fm-3]
    2: energy density [Mev fm-3]
    Conversion factor: 1 [Mev fm-3] = 1.323x10^-6 [Msol c2 km-3]
    (see above)
    
    Also does a little calculation for sound speed (cs) to be used 
    in calculation of tidal Love number
    """
    
    src = np.loadtxt(pathname)#, comments='#')
    P, e = src[:,1], src[:,2]
    p, e = P*nuc_to_astro, e*nuc_to_astro
    cs = []
    for i in range(len(p)-1):
        cs_temp = (p[i+1]-p[i])/(e[i+1]-e[i])
        cs.append(cs_temp)
    return (p,e,cs)
    
def make_derivs_func(pathname):
    pdata, edata, csdata = read_eos(pathname)
    eos = interp1d(pdata,edata,fill_value='extrapolate')
    c_eos = interp1d(pdata[0:-1],csdata,fill_value='extrapolate')    
    def derivs(p,y):
        """
        e: e_of_P function
        """
        e = eos(p)
        cs = max(c_eos(p),1e-6) # Just to make sure there are no zeroes- do not worry about it
        x,m,Y = y
        
        F = (np.sqrt(x)/(np.sqrt(x)-(2*m))) * (1-(4*np.pi*x*(e-p)))
        Q = (1/x)*(((4*np.pi*x**(3/2))/(np.sqrt(x)-(2*m)))*(5*e + 9*p + ((e+p)/(cs))) - 
             6*(np.sqrt(x)/(np.sqrt(x)-(2*m))) - ((4*(m+4*np.pi*p*(x**(3/2)))**2)/(np.sqrt(x)-2*m)**2))
        
        dxdp = -2*x*(np.sqrt(x)-(2*m))/((e+p)*(m+(4*np.pi*p*x**(3/2))))
        dmdp = -((4*np.pi*e*x**(3/2))*(np.sqrt(x)-2*m))/((e+p)*(m+(4*np.pi*p*x**(3/2))))
        dydp = ((Y**2+(Y*F)+(x*Q))*(np.sqrt(x)-2*m))/((e+p)*(m+4*np.pi*p*x**(3/2)))
        return dxdp,dmdp,dydp
    
    return derivs

def solve_tov(p_init,pathname):
    """
    Inputs:
    - p,e,cs: functions for EOS
    - p_init: Value of central pressure
    Note: This seems to work best if you use RK23 and let the tolerances be the
    default values. If you make atol for Y too small (O(1e-2)), you start to get 
    a lot of NaN values. If you make it too big, downstream quantities like k2 
    get too zig-zaggy. LSODA runs into a lot of "divide by zero" singularities :(
    """
    
    derivs = make_derivs_func(pathname)

    p_range = [p_init,1e-14]
    y0 = [1e-14,1e-14,2]
    sol = solve_ivp(derivs, p_range, y0, dense_output=True, 
                    #atol=(1e-6,1e-6,1e-2), rtol=1e-6, 
                    method='RK23')
    return sol

In [11]:
import traceback
import random

# Some things that do not need to change:
# Range of central pressures to start with (nuclear units)
P_vals = np.geomspace(5,30000,70)
# Number of EOS files to do calculations for:
n = 5
eos_path = 'EOS_files_nsat/'
eos_files = [699,278,571,1092,1357]#random.sample(range(2000), n)

#radmass = {}
for fnum in eos_files:
    start_time=time.time() # Just a little thing to show how long each EOS takes
    #radmass[fnum] = []
    radmass = []
    eospath = 'EOS_files_nsat/%d.dat'%fnum
    
    for P in P_vals:
        try:
            init_p = P*nuc_to_astro
            sol = solve_tov(init_p, eospath)
            mass, radius, yR = sol.y[1][-1]/rs, np.sqrt(sol.y[0][-1]), sol.y[2][-1]
            
            # compactness (C) = GM/Rc^2
            C = mass*rs / radius     # Compactness looks good!
            #Calculate k2:
            # For convenience, factors of yR that go into k2 (math is hard):
            y3,y5,y11 = 3*yR, 5*yR, 11*yR
            # Similarly, factors of C:
            c2 = 2*C
            term1 = (8/5)*(C**5)*((1-c2)**2)*(2-yR+(c2*(yR-1)))
            term2 = 2*C*(6-y3+3*C*(y5-8))
            term3 = 4*(C**3)*(13-y11+C*(y3-2)+2*(C**2)*(1+yR))
            term4 = (3*((1-c2)**2))*(2-yR+c2*(yR-1))*np.log(1-c2)
            k2 = term1*((term2+term3+term4)**(-1))
            
            Lambda = (2/3)*k2*(C**-5)
            
            #radmass[fnum].append([radius, mass, Lambda, k2, C])
            radmass.append([radius, mass, Lambda, k2, C])
        except:
            print('An error has occurred. Turn on traceback.')
            #traceback.print_exc()
    radmass = np.array(radmass)
    
    # Bonus: find the index where max(mass) occurs and cut it off there.
    indmax = np.where(radmass[:,1]==max(radmass[:,1]))[-1][0]
    radmass = radmass[:indmax]
    np.save('results/%d.npy'%fnum, radmass)
    print('EOS',fnum,
          'took about %1.2f seconds, and resulted in a maximum mass of %1.2f M_sun.'%(time.time()-start_time,max(radmass[:,1])))

EOS 699 took about 5.27 seconds, and resulted in a maximum mass of 2.06 M_sun.
EOS 278 took about 5.45 seconds, and resulted in a maximum mass of 1.92 M_sun.
EOS 571 took about 5.37 seconds, and resulted in a maximum mass of 2.17 M_sun.
EOS 1092 took about 5.39 seconds, and resulted in a maximum mass of 2.04 M_sun.
EOS 1357 took about 5.37 seconds, and resulted in a maximum mass of 2.63 M_sun.
